In [84]:
!pip install ipywidgets==7.0.0 --quiet
!pip install sagemaker==2.169.0

In [85]:
#Load libraries

import pandas as pd
import json
import boto3
import pathlib
import io
import sagemaker
from time import gmtime, strftime, sleep
from sagemaker.deserializers import CSVDeserializer
from sagemaker.serializers import CSVSerializer

from sagemaker.workflow.pipeline_context import PipelineSession
from sagemaker.xgboost.estimator import XGBoost
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import (
    ProcessingInput, 
    ProcessingOutput, 
    ScriptProcessor
)
from sagemaker.inputs import TrainingInput

from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.steps import (
    ProcessingStep, 
    TrainingStep, 
    CreateModelStep
)
from sagemaker.workflow.check_job_config import CheckJobConfig
from sagemaker.workflow.parameters import (
    ParameterInteger, 
    ParameterFloat, 
    ParameterString, 
    ParameterBoolean
)
from sagemaker.workflow.clarify_check_step import (
    ModelBiasCheckConfig, 
    ClarifyCheckStep, 
    ModelExplainabilityCheckConfig
)
from sagemaker import Model
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.model_step import ModelStep
from sagemaker.workflow.fail_step import FailStep
from sagemaker.workflow.conditions import (
    ConditionGreaterThan,
    ConditionGreaterThanOrEqualTo
)
from sagemaker.workflow.pipeline_experiment_config import PipelineExperimentConfig
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import (
    Join,
    JsonGet
)
from sagemaker.workflow.lambda_step import (
    LambdaStep,
    LambdaOutput,
    LambdaOutputTypeEnum,
)
from sagemaker.lambda_helper import Lambda

from sagemaker.model_metrics import (
    MetricsSource, 
    ModelMetrics, 
    FileSource
)
from sagemaker.drift_check_baselines import DriftCheckBaselines

from sagemaker.image_uris import retrieve

sagemaker.__version__

'2.169.0'

In [86]:
%store -r 

%store

try:
    initialized
except NameError:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN 00-start-here notebook   ")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++")

Stored variables and their in-db values:
baseline_s3_url                          -> 's3://sagemaker-zara-blouses-generation/loading_da
bucket                                   -> 'sagemaker-studio-eu-central-1-567821811420'
bucket_name                              -> 'sagemaker-zara-blouses-generation'
bucket_prefix                            -> 'loading_dataset'
customers_count                          -> 10000
customers_feature_group_name             -> 'fscw-customers-06-21-08-51'
docker_image_name                        -> '492215442770.dkr.ecr.eu-central-1.amazonaws.com/s
domain_id                                -> 'd-ivd5gnez0yil'
evaluation_s3_url                        -> 's3://sagemaker-zara-blouses-generation/loading_da
framework_version                        -> '1.3-1'
initialized                              -> True
model_package_group_name                 -> 'zara-course-project-model-group'
orders_count                             -> 100000
orders_feature_group_name    

SET CONSTANTS

In [5]:
# Set names of pipeline objects
project = "zara-course-project"

pipeline_name = f"{project}-pipeline"
pipeline_model_name = f"{project}-model-txt2img"
model_package_group_name = f"{project}-model-group"
endpoint_config_name = f"{project}-endpoint-config"
endpoint_name = f"{project}-endpoint"

In [6]:
# Set instance types and counts
process_instance_type = "ml.g4dn.8xlarge"
train_instance_count = 1
train_instance_type = "ml.g4dn.8xlarge"

In [7]:
# Set S3 urls for processed data
train_s3_url = f"s3://{bucket_name}/{bucket_prefix}/pipeline_test/train"
output_s3_url = f"s3://{bucket_name}/{bucket_prefix}/pipeline_test/output"

In [8]:
%store output_s3_url

Stored 'output_s3_url' (str)


In [9]:
#Supress default INFO logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

In [10]:
import sagemaker 
role = sagemaker.get_execution_role()
sagemaker_session = sagemaker.session.Session()

In [ ]:
%store -r s3_uri_raw
s3_uri_raw

'sagemaker-zara-blouses-generation/loading_dataset/zara_images_initial/'

In [ ]:
# Set processing instance type
process_instance_type_param = ParameterString(
    name="ProcessingInstanceType",
    default_value=process_instance_type,
)

# Set training instance type
train_instance_type_param = ParameterString(
    name="TrainingInstanceType",
    default_value=train_instance_type,
)

# Set training instance count
train_instance_count_param = ParameterInteger(
    name="TrainingInstanceCount",
    default_value=train_instance_count
)

# Set model approval param
model_approval_status_param = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval"
)

# Set S3 url for input dataset
input_s3_url_param = ParameterString(
    name="InputDataUrl",
    default_value=s3_uri_raw,
)

In [13]:
session = PipelineSession()

Preprocessing step

In [14]:
%%writefile preprocessing.py

import pandas as pd
import numpy as np
import argparse
import os
from transformers import AutoFeatureExtractor, SegformerForSemanticSegmentation
from PIL import Image
import requests
import matplotlib.pyplot as plt
import torch.nn as nn
import os
import numpy as np
import torchvision.utils as vutils



def _parse_args():
    
    parser = argparse.ArgumentParser()
    # Data, model, and output directories
    # model_dir is always passed in from SageMaker. By default this is a S3 path under the default bucket.
    parser.add_argument('--filepath', type=str, default='/opt/ml/processing/input/')
    parser.add_argument('--outputpath', type=str, default='/opt/ml/processing/output/train/')
    
    return parser.parse_known_args()


if __name__=="__main__":
    extractor = AutoFeatureExtractor.from_pretrained("mattmdjaga/segformer_b2_clothes")
    model = SegformerForSemanticSegmentation.from_pretrained("mattmdjaga/segformer_b2_clothes")

    input_folder_path = args.filepath
    output_folder_path = args.outputpath

    # Iterate over the images in the input folder
    for filename in os.listdir(input_folder_path):
        if filename.endswith('.jpg') or filename.endswith('.png'):  # Update with supported image extensions
            # Read the image
            image_path = os.path.join(input_folder_path, filename)
            image = Image.open(image_path)

            inputs = extractor(images=image, return_tensors="pt")
            outputs = model(**inputs)
            logits = outputs.logits.cpu()

            upsampled_logits = nn.functional.interpolate(
                logits,
                size=image.size[::-1],
                mode="bilinear",
                align_corners=False,
            )

            pred_seg = upsampled_logits.argmax(dim=1)[0]

            # Extract the upper cloth mask
            upper_cloth_label = 4  # Replace with the actual label index for the upper cloth
            upper_cloth_mask = pred_seg == upper_cloth_label

            # Convert the tensor to a NumPy array
            img_upper_cloth_mask = upper_cloth_mask.numpy().astype(np.uint8) * 255

            # Convert the NumPy array to a PIL Image
            image_pil = Image.fromarray(img_upper_cloth_mask)

            # Specify the output image file path with the .jpg extension
            output_image_path = os.path.join(output_folder_path, filename)
            output_image_path = output_image_path.replace('.png', '.jpg')

            # Save the image as JPEG
            image_pil.save(output_image_path)
            
            
            
    image_folder = '/content/drive/MyDrive/images_zara'
    mask_folder = '/content/drive/MyDrive/images_zara_mask'
    output_folder_path = '/content/drive/MyDrive/images_zara_final/'

    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    image_files = os.listdir(image_folder)

    for image_file in image_files:
        # Load the image and mask
        image_path = os.path.join(image_folder, image_file)
        mask_path = os.path.join(mask_folder, image_file)  # Assuming mask filenames are the same as image filenames
        image = cv2.imread(image_path)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # Apply the mask as a binary filter
        masked_image = cv2.bitwise_and(image, image, mask=mask)

        output_image_path = os.path.join(output_folder_path, image_file)
        output_image_path = output_image_path.replace('.png', '.jpg')

        if masked_image is None:
          print("Error: Failed to load the image.")
          continue

        # Save the masked image
        cv2.imwrite(output_image_path, masked_image)


Overwriting preprocessing.py


In [ ]:
sklearn_processor = SKLearnProcessor(
        framework_version="0.23-1",
        role=sm_role,
        instance_type=process_instance_type_param,
        instance_count=1,
        base_job_name=f"{pipeline_name}/preprocess",
        sagemaker_session=session,
    )
    
processing_inputs=[
    ProcessingInput(source=input_s3_url_param, destination="/opt/ml/processing/input")
]

processing_outputs=[
    ProcessingOutput(output_name="train_data", source="/opt/ml/processing/output/train", 
                     destination=train_s3_url)
]

processor_args = sklearn_processor.run(
    inputs=processing_inputs,
    outputs=processing_outputs,
    code='preprocessing.py',
    # arguments = ['arg1', 'arg2'],
)
    
# Define processing step
step_process = ProcessingStep(
    name=f"{pipeline_name}-preprocess-data",
    step_args=processor_args,
)

---

<a id="train_step"></a>

# 2. Create modeling step

In [16]:
%store -r s3_model_data
s3_model_data

'sagemaker-zara-blouses-generation/loading_dataset/model/model.tar.gz'

In [77]:
model_id, model_version = "model-txt2img-stabilityai-stable-diffusion-v2-1-base", "*"

In [18]:
train_s3_url

's3://sagemaker-zara-blouses-generation/loading_dataset/pipeline_test/train'

In [19]:
s3_model_data = 's3://' + s3_model_data

In [72]:
train_source_uri

's3://jumpstart-cache-prod-eu-central-1/source-directory-tarballs/stabilityai/transfer_learning/txt2img/prepack/v1.0.3/sourcedir.tar.gz'

In [92]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters, instance_types
from sagemaker.model import Model
from sagemaker.predictor import Predictor
from sagemaker.utils import name_from_base
from sagemaker import hyperparameters

hyperparameters = hyperparameters.retrieve_default(
    model_id=model_id, model_version=model_version
)

# Tested with ml.g4dn.2xlarge (16GB GPU memory) and ml.g5.2xlarge (24GB GPU memory) instances. Other instances may work as well.
# If ml.g5.2xlarge instance type is available, please change the following instance type to speed up training.
training_instance_type = instance_types.retrieve_default(
    region=None,
    model_id=model_id,
    model_version=model_version,
    scope="training"
)

# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,  # automatically inferred from model_id
    model_id=model_id,
    model_version=model_version,
    image_scope="training",
    instance_type='ml.g4dn.xlarge',
)

train_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="training"
)

train_model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="training"
)

# Instantiate an XGBoost estimator object
sd_estimator = sagemaker.estimator.Estimator(
    role=sm_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",  # Entry-point file in source_dir and present in train_source_uri.
    instance_count=1,
    instance_type='ml.g4dn.xlarge',
    max_run=360000,
    output_path=output_s3_url,
    hyperparameters=hyperparameters,
    # base_job_name=training_job_name,
)

# Define training step
step_train = TrainingStep(
    name=f"{pipeline_name}-train",
    estimator=sd_estimator,
    inputs={
        "train": TrainingInput(s3_data="s3://sagemaker-zara-blouses-generation/loading_dataset/zara_images_final/")
    }
)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.


In [91]:
sd_estimator.fit({"training": "

In [ ]:
s3://sagemaker-zara-blouses-generation/loading_dataset/zara_images_final/"}, logs=True)

---

<a id="appr_model_reg_step"></a>

# 4. Create approve model and save to model registry steps

In [93]:
from sagemaker.workflow.conditions import (
    ConditionGreaterThanOrEqualTo,
)
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)

Add new input parameter for the model registration step:

In [94]:
model_approval_status = ParameterString(
    name="ModelApprovalStatus",
    default_value="PendingManualApproval",  # ModelApprovalStatus can be set to a default of "Approved" if you don't want manual approval.
)

Create register model step:

In [95]:
model = Model(
    image_uri=s3_model_data,        
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    name=f"zara-project-model",
    sagemaker_session=session,
    role=sm_role,
)

register_args = model.register(
    content_types=["image/png"],
    response_types=["image/png"],
    inference_instances=["ml.g4dn.xlarge"],
    transform_instances=["ml.g4dn.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status_param,
)

step_register = ModelStep(
    name=f"{pipeline_name}-register",
    step_args=register_args
)

/opt/conda/lib/python3.7/site-packages/sagemaker/workflow/pipeline_context.py:289: UserWarning: Running within a PipelineSession, there will be No Wait, No Logs, and No Job being started.
  UserWarning,


Create condition step for **accuracy above 0.8**:

## Create the forth and final iteration of the Pipeline (updating the definition)

We will update the pipeline the final approve model (contidion) and save model steps, resulting in a pipeline with 5 steps: data prep, training, evaluation, approval, save to registry steps.

In [96]:
pipeline = Pipeline(
        name=pipeline_name,
        parameters=[
            process_instance_type_param,
            training_instance_type,
            model_approval_status,
        ],
        steps=[step_train, step_register],
        sagemaker_session=sagemaker_session,
    )

In [97]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:eu-central-1:567821811420:pipeline/zara-course-project-pipeline',
 'ResponseMetadata': {'RequestId': '1b6fccd7-81e5-4a76-86b4-0054cd23ba30',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1b6fccd7-81e5-4a76-86b4-0054cd23ba30',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '99',
   'date': 'Fri, 30 Jun 2023 07:26:22 GMT'},
  'RetryAttempts': 0}}

Let's start final execution:

In [69]:
execution = pipeline.start()

<a id='end_creation_pipe'></a>
    
# 5. End of pipeline creation

With the caches all should be faster now.

Let's get the final result of the pipeline. Read evaluation report:

#### Run again with caches and changing input parameters:

In [98]:
# Obs.: If we want to override the input parameters with other ones:

execution = pipeline.start(
    parameters=dict(
        ProcessingInstanceType="ml.g4dn.xlarge",
        ModelApprovalStatus="Approved", # Would approve automatically
    )
)

In [ ]:
# # Obs.: if we wanted to stop pipeline execution:
# execution.stop()

In [ ]:
# # Obs.: if we wanted to delete the whole pipeline:
# pipeline.delete()

In [54]:
import json

my_vars = {
    "bucket": bucket_name, 
    "prefix": bucket_prefix, 
    "region": region,
    "s3_uri_raw": s3_uri_raw,
    "role": rol
    }

with open("../my-solution-vars.json", "w") as f:
    f.write(json.dumps(my_vars))


Let's put the whole pipeline code into a python script:

In [100]:
%%writefile pipeline_definition.py
import os
import json
from time import strftime, gmtime

import sagemaker
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import (
    ProcessingStep,
    TrainingStep,
)
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    ScriptProcessor,
)
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.pipeline_experiment_config import PipelineExperimentConfig
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import CacheConfig
from sagemaker.workflow.conditions import (
    ConditionGreaterThanOrEqualTo,
)
from sagemaker.workflow.condition_step import (
    ConditionStep,
    JsonGet,
)
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)
from sagemaker import hyperparameters


BASE_DIR = os.path.dirname(os.path.realpath(__file__))

def get_my_solutions_vars():
    vars_path = os.path.join(".", "pipelines", "my_labs_solutions", "my-solution-vars.json")

    with open(vars_path, "rb") as f:
        my_vars = json.loads(f.read())
        
    return my_vars

def get_pipeline(region,
                 role=None,
                 default_bucket=None,
                 model_package_group_name="MLOpsZaraProject",  # Choose any name
                 pipeline_name="MLOpsFinalChurnMLPipeline",  # You can find your pipeline name in the Studio UI (project -> Pipelines -> name)
                 base_job_prefix="ZazaJob",  # Choose any name
                ) -> Pipeline:
    
    # Get config vars
    my_vars = get_my_solutions_vars()
    bucket = my_vars["bucket"]
    prefix = my_vars["prefix"]
    region = my_vars["region"]
    s3_uri_raw = my_vars["s3_uri_raw"]
    role = my_vars["role"]

    sagemaker_session = sagemaker.session.Session()

    # Parameters for data preparation step
    input_data = ParameterString(
        name="InputDataUrl",
        default_value=s3uri_raw # S3 URI where we stored the raw data
    )
    # Add an input parameter to define the training instance type
    training_instance_type = ParameterString(
        name="TrainingInstanceType", default_value="ml.g4dn.xlarge"
    )
    model_approval_status = ParameterString(
        name="ModelApprovalStatus",
        default_value="PendingManualApproval",  # ModelApprovalStatus can be set to a default of "Approved" if you don't want manual approval.
    )


    # Cache for 30 minutes
    cache_config = CacheConfig(enable_caching=True, expire_after="T30m")

    training_instance_type = instance_types.retrieve_default(
        region=None,
        model_id=model_id,
        model_version=model_version,
        scope="training"
    )
    
    hyperparameters = hyperparameters.retrieve_default(
        model_id=train_model_id, model_version=train_model_version
    )

    # Retrieve the docker image
    train_image_uri = image_uris.retrieve(
        region=None,
        framework=None,  # automatically inferred from model_id
        model_id=model_id,
        model_version=model_version,
        image_scope="training",
        instance_type='ml.g4dn.xlarge',
    )

    train_source_uri = script_uris.retrieve(
        model_id=model_id, model_version=model_version, script_scope="training"
    )

    train_model_uri = model_uris.retrieve(
        model_id=model_id, model_version=model_version, model_scope="training"
    )

    # Instantiate an XGBoost estimator object
    sd_estimator = sagemaker.estimator.Estimator(
        role=sm_role,
        image_uri=train_image_uri,
        source_dir=train_source_uri,
        model_uri=train_model_uri,
        entry_point="transfer_learning.py",  # Entry-point file in source_dir and present in train_source_uri.
        instance_count=1,
        instance_type='ml.g4dn.xlarge',
        max_run=360000,
        output_path=output_s3_url,
        hyperparameters=hyperparameters,
        # base_job_name=training_job_name,
    )

    # Define training step
    step_train = TrainingStep(
        name=f"{pipeline_name}-train",
        estimator=sd_estimator,
        inputs={
            "train": TrainingInput(s3_data="s3://sagemaker-zara-blouses-generation/loading_dataset/zara_images_final")
        }
    ) 
    # Register model step that will be conditionally executed
    model = Model(
        image_uri=train_image_uri,        
        model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
        name=f"zara-project-model",
        sagemaker_session=session,
        role=sm_role,
    )

    register_args = model.register(
        content_types=["image/png"],
        response_types=["image/png"],
        inference_instances=["ml.g4dn.xlarge"],
        transform_instances=["ml.g4dn.xlarge"],
        model_package_group_name=model_package_group_name,
        approval_status=model_approval_status_param,
    )

    step_register = ModelStep(
        name=f"{pipeline_name}-register",
        step_args=register_args
    )



    # Experiment configs
    create_date = lambda: strftime("%Y-%m-%d-%H-%M-%S", gmtime())

    experiment_name=f"pipeline-customer-churn-prediction-xgboost-{create_date()}"
    trial_name=f"pipeline-framework-trial-{create_date()}"

    pipeline_experiment_config = PipelineExperimentConfig(
        experiment_name = experiment_name,
        trial_name = trial_name
    )


    pipeline = Pipeline(
            name=pipeline_name,
            parameters=[
                input_data,
                processing_instance_type,
                processing_instance_count,
                training_instance_type,
                model_approval_status,
            ],
            steps=[step_process, step_train, step_eval, step_cond],
            sagemaker_session=sagemaker_session,
        )
    
    return pipeline


Overwriting pipeline_definition.py
